In [6]:
import pandas as pd
import numpy as np

In [1]:
!pip install skl2onnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3

In [2]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.5 MB/s eta 0:00:00


In [9]:
import onnxruntime as rt

def predict_with_onnx(input_data):
    sess = rt.InferenceSession("adaboost_loan_model.onnx")
    input_name = sess.get_inputs()[0].name
    input_data = input_data.astype(np.float32)
    pred_onx = sess.run(None, {input_name: input_data})
    return pred_onx[0]

def predict_loan_approval(gender, married, dependents, education, self_employed,
                         applicant_income, coapplicant_income, loan_amount,
                         loan_term, credit_history, property_area):

    gender_map = {'Male': 1, 'Female': 0}
    married_map = {'Yes': 1, 'No': 0}
    dependents_map = {'0': 0, '1': 1, '2': 2, '3+': 3}
    education_map = {'Graduate': 1, 'Not Graduate': 0}
    self_employed_map = {'Yes': 1, 'No': 0}
    property_area_map = {'Urban': 2, 'Semiurban': 1, 'Rural': 0}

    input_data = np.array([[
        gender_map[gender],
        married_map[married],
        dependents_map[dependents],
        education_map[education],
        self_employed_map[self_employed],
        applicant_income,
        coapplicant_income,
        loan_amount,
        loan_term,
        credit_history,
        property_area_map[property_area]
    ]])

    prediction = predict_with_onnx(input_data)
    return prediction[0]



In [8]:
example_prediction = predict_loan_approval(
    gender='Male',
    married='Yes',
    dependents='3+',
    education='Graduate',
    self_employed='No',
    applicant_income=0,
    coapplicant_income=0,
    loan_amount=200,
    loan_term=360,
    credit_history=1,
    property_area='Rural'
)

print("Loan Approval Prediction:", example_prediction)


Loan Approval Prediction: 1


In [7]:
example_prediction = predict_loan_approval('Male', 'Yes', '3+', 'Graduate', 'No', 0, 0, 200, 360, 1, 'Rural')
print(example_prediction)

1
